In [1]:
%pylab
from landlab import RasterModelGrid
from landlab.plot.imshow import imshow_grid_at_node
from landlab.components import PresFlowNetwork, MeltCreep
import numpy as np
from matplotlib import colors,  cm
import matplotlib.animation as animation

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
mg = RasterModelGrid((25,30),500)
junc_elev = mg.add_zeros('node', 'junction__elevation')
R = mg.add_zeros('node', 'input__discharge')
#d_h = mg.add_ones('link', 'hydraulic__diameter')
thickness = 500.*np.ones(mg.number_of_nodes)
Z = mg.add_field('node', 'ice__thickness', thickness)
#a = mg.add_ones('link', 'head_loss__exponent')
#f = mg.add_ones('link', 'friction__factor')
#h = np.random.rand(mg.number_of_nodes)
h = mg.add_zeros('node', 'hydraulic__head')
Q = mg.add_zeros('link', 'conduit__discharge')
#net_node_flux = mg.add_ones('node', 'net_node_flux', noclobber=False)
#set boundary node head

#add input to node 310
moulin_Q=1.
mg.at_node['input__discharge'][305]=moulin_Q
#set heads at edges
h[mg.nodes_at_left_edge] = 400.
h[mg.nodes_at_right_edge] = 300.
h[mg.nodes_at_top_edge] = 0.
h[mg.nodes_at_bottom_edge] = 0.
mg.set_closed_boundaries_at_grid_edges(False,True,True,True)
Q[mg.active_links] = 1.*0.1
n_core = mg.number_of_core_nodes
links = mg.links_at_node
print("Number of links = ", mg.number_of_links)
print("Number of nodes = ", mg.number_of_nodes)
print("Number of active links = ", mg.number_of_active_links)
print("Number of core nodes = ", mg.number_of_core_nodes)

d_h = mg.add_zeros('link','hydraulic__diameter')
mg.at_link['hydraulic__diameter'][mg.active_links]= 0.5*np.random.rand(mg.number_of_active_links)




#code added for fsolve algorithm
#dhdx = mg.add_zeros('link', 'hydraulic_head_gradient', noclobber=False)
#net_node_flux = mg.add_ones('node', 'net_node_flux', noclobber=False)


pfn = PresFlowNetwork(mg)
pfn.run_one_step()
mc = MeltCreep(mg, dt=2500.)

Number of links =  1445
Number of nodes =  750
Number of active links =  1260
Number of core nodes =  644
Number of iterations = 1 tolerance = 0.247278373905  iteration time= 0.19241547584533691


/home/mcoving/Data/github/landlab/landlab/components/conduit_networks/pressurized_flow.py:155: RuntimeWarning: divide by zero encountered in true_divide
  self.r = 8*f*L/(g*r_s**2.*d_h**5.)


Number of iterations = 2 tolerance = 0.224365186594  iteration time= 0.20879793167114258
Number of iterations = 3 tolerance = 0.155887663551  iteration time= 0.18712425231933594
Number of iterations = 4 tolerance = 0.138214008714  iteration time= 0.19321227073669434
Number of iterations = 5 tolerance = 0.072623052848  iteration time= 0.1893014907836914
Number of iterations = 6 tolerance = 0.0272770920935  iteration time= 0.19333767890930176
Number of iterations = 7 tolerance = 0.0146059697607  iteration time= 0.1918950080871582
Number of iterations = 8 tolerance = 0.0443590109056  iteration time= 0.19707393646240234
Number of iterations = 9 tolerance = 0.0234006098079  iteration time= 0.18820977210998535
Number of iterations = 10 tolerance = 0.0116840115217  iteration time= 0.19096732139587402
Number of iterations = 11 tolerance = 0.00580951249822  iteration time= 0.18591666221618652
Number of iterations = 12 tolerance = 0.00284078710223  iteration time= 0.19277381896972656
Number of i

In [9]:
type(round(mg.number_of_cell_columns*15))

int

In [8]:
from PIL import Image
from PIL import ImageDraw

def plot_links(grid, value_name, autoscale=True, vmin=0., vmax=0., cmap_name='viridis', magnitude= False, lw=5, pixels_per_node=15, x_name='X', y_name='Y', var_name=''):#, logvalues=False ):
    pixel_xscaling_factor = pixels_per_node/grid.dx
    pixel_yscaling_factor = pixels_per_node/grid.dy
    im = Image.new('RGBA', ( int(round(grid.number_of_cell_columns*pixel_xscaling_factor)), int(round(grid.number_of_cell_rows*pixel_yscaling_factor)) ), (255,255,255,255))
    draw = ImageDraw.Draw(im)

    link_head_x = grid.node_x[grid.node_at_link_head]
    link_head_y = grid.node_y[grid.node_at_link_head]
    link_tail_x = grid.node_x[grid.node_at_link_tail]
    link_tail_y = grid.node_y[grid.node_at_link_tail]       
    if magnitude:
        values = abs(grid.at_link[value_name])
    else:
        values = grid.at_link[value_name]
    #if logvalues:
    #    values = np.log10(abs(grid.at_link[value_name]))
    
    
    #Normalize color values
    if autoscale:
        cnorm = colors.Normalize()
        cnorm.autoscale(values)
    else:
        cnorm = colors.Normalize(vmin,vmax)
    scalarMap = cm.ScalarMappable(norm=cnorm, cmap = get_cmap(cmap_name))
    scalarMap.set_array(values)
    #set_cmap()
    for i, value in enumerate(values):
        draw.line( ((link_head_x[i]*pixel_xscaling_factor,link_head_y[i]*pixel_yscaling_factor),(link_tail_x[i]*pixel_xscaling_factor,link_tail_y[i]*pixel_yscaling_factor)),fill=scalarMap.to_rgba(value,bytes=True),width=lw) 
        #xs = [link_head_x[i],link_tail_x[i]]
        #ys = [link_head_y[i],link_tail_y[i]]
        #img = plot(xs,ys,lw=lw,color=scalarMap.to_rgba(value))
    
    imshow(np.asarray(im), origin='lower', extent=(0,grid.number_of_cell_columns,0,grid.number_of_cell_rows))
    cb = colorbar(scalarMap)
    cb.ax.set_ylabel(var_name)
    xlabel(x_name)
    ylabel(y_name)


In [10]:
nsteps = 1000
step_start = 1
fig1, axs = subplots(2,2,figsize=(10,6))
every = 50
for step in arange(nsteps)+step_start:
    print("step =",step, " avg d_h=",mg.at_link['hydraulic__diameter'].mean())
    pfn.run_one_step()    
    #mg.calc_grad_at_link(h, out=dhdx)
    #h = fsolve(network_residuals, h[mg.core_nodes])
    #Q = np.sign(dhdx)*np.sqrt(np.fabs(dhdx))
    mc.run_one_step()
    if (step % every)==0: #make an animation frame
        subplot(2,2,1)
        imshow_grid_at_node(mg, h, var_name='Head ($m$)', cmap='viridis',vmin=300,vmax=400)
        #plot(mg.at_node['hydraulic__head'][mg.core_nodes])
        #ylim([300,400])
        subplot(2,2,2)
        plot_links(mg, 'conduit__discharge', magnitude=True, var_name='Discharge ($m^3/s$)', vmin=0., vmax=1, autoscale=False)
        #plot(mg.at_link['conduit__discharge'][mg.active_links])
        #ylabel('')
        #ylim([0,1.5])
        subplot(2,2,3)
        plot_links(mg, 'hydraulic__diameter', var_name='Hydraulic diameter ($m$)', autoscale=False, vmin=0, vmax=1.25)
        subplot(2,2,4)#, position=[0.2,0.2,0.9,0.9])
        plot(mg.at_link['hydraulic__diameter'][mg.active_links])
        xlabel('Link number')
        ylabel('Hydraulic diameter ($m$)')
        ylim(0,1.25)
        #ylim([0.5,1.5])
        image_name = 'heads_and_discharge'+str(step).zfill(6)+'.png'
        #tight_layout()
        subplots_adjust(wspace=0.3)
        savefig(image_name)
        fig1.clf()

#im_ani = animation.ArtistAnimation(fig1,ims)

step = 1  avg d_h= 0.231551431911
Number of iterations = 1 tolerance = 0.000429020453429  iteration time= 0.19464707374572754
Number of iterations = 2 tolerance = 1.88287294845e-06  iteration time= 0.1948716640472412
max ddh =  0.00125144230361
step = 2  avg d_h= 0.231590077909
Number of iterations = 1 tolerance = 0.000417285082883  iteration time= 0.19008970260620117
Number of iterations = 2 tolerance = 1.94433657661e-06  iteration time= 0.1947009563446045
max ddh =  0.00122859048163
step = 3  avg d_h= 0.231627308469
Number of iterations = 1 tolerance = 0.000409476471283  iteration time= 0.21875476837158203
Number of iterations = 2 tolerance = 2.01007251088e-06  iteration time= 0.20255351066589355
max ddh =  0.00120646224116
step = 4  avg d_h= 0.231663188769
Number of iterations = 1 tolerance = 0.000401863318345  iteration time= 0.2092294692993164
Number of iterations = 2 tolerance = 2.08147857543e-06  iteration time= 0.19417381286621094
max ddh =  0.00118502716355
step = 5  avg d_h= 

Number of iterations = 2 tolerance = 5.68075950462e-07  iteration time= 0.19249987602233887
max ddh =  0.0007904518921
step = 35  avg d_h= 0.232331950156
Number of iterations = 1 tolerance = 0.000302803131269  iteration time= 0.18743896484375
Number of iterations = 2 tolerance = 5.38400827965e-07  iteration time= 0.19558048248291016
max ddh =  0.000782146833361
step = 36  avg d_h= 0.232343914546
Number of iterations = 1 tolerance = 0.000301483203917  iteration time= 0.18837261199951172
Number of iterations = 2 tolerance = 5.10822925222e-07  iteration time= 0.2061316967010498
max ddh =  0.00077397985177
step = 37  avg d_h= 0.232355470141
Number of iterations = 1 tolerance = 0.000300183163058  iteration time= 0.19094324111938477
Number of iterations = 2 tolerance = 4.85175983777e-07  iteration time= 0.21933698654174805
max ddh =  0.000765948302861
step = 38  avg d_h= 0.232366628505
Number of iterations = 1 tolerance = 0.000298902369051  iteration time= 0.2177867889404297
Number of iterat

Number of iterations = 2 tolerance = 4.23910883041e-07  iteration time= 0.19960904121398926
max ddh =  0.000576511206786
step = 68  avg d_h= 0.232560125984
Number of iterations = 1 tolerance = 0.000267152514596  iteration time= 0.1946885585784912
Number of iterations = 2 tolerance = 4.42497844907e-07  iteration time= 0.20298385620117188
max ddh =  0.000571585742485
step = 69  avg d_h= 0.232562975173
Number of iterations = 1 tolerance = 0.000266262427897  iteration time= 0.2176342010498047
Number of iterations = 2 tolerance = 4.63223372419e-07  iteration time= 0.19783973693847656
max ddh =  0.000566732134053
step = 70  avg d_h= 0.232565647232
Number of iterations = 1 tolerance = 0.000265380664439  iteration time= 0.20286822319030762
Number of iterations = 2 tolerance = 4.86452761172e-07  iteration time= 0.20235157012939453
max ddh =  0.000561949012813
step = 71  avg d_h= 0.232568146195
Number of iterations = 1 tolerance = 0.000264507062307  iteration time= 0.19425368309020996
Number of 

step = 101  avg d_h= 0.232579131433
Number of iterations = 1 tolerance = 0.000241264566223  iteration time= 0.18978595733642578
Number of iterations = 2 tolerance = 9.19883825438e-07  iteration time= 0.19518256187438965
max ddh =  0.000496747354751
step = 102  avg d_h= 0.232577821023
Number of iterations = 1 tolerance = 0.000240572750823  iteration time= 0.1886131763458252
Number of iterations = 2 tolerance = 7.19517866847e-07  iteration time= 0.19806909561157227
max ddh =  0.000495096739619
step = 103  avg d_h= 0.23257642694
Number of iterations = 1 tolerance = 0.000239885441072  iteration time= 0.2090282440185547
Number of iterations = 2 tolerance = 5.82888467923e-07  iteration time= 0.20129704475402832
max ddh =  0.000493459887506
step = 104  avg d_h= 0.232574951098
Number of iterations = 1 tolerance = 0.000239202550067  iteration time= 0.19114899635314941
Number of iterations = 2 tolerance = 4.85787281466e-07  iteration time= 0.20597577095031738
max ddh =  0.000491836561408
step = 

Number of iterations = 2 tolerance = 3.92601909001e-07  iteration time= 0.19920825958251953
max ddh =  0.000448475183589
step = 135  avg d_h= 0.232497590294
Number of iterations = 1 tolerance = 0.00021998089147  iteration time= 0.1865556240081787
Number of iterations = 2 tolerance = 4.44926013938e-07  iteration time= 0.2177290916442871
max ddh =  0.000447181874207
step = 136  avg d_h= 0.232494322411
Number of iterations = 1 tolerance = 0.000219414451434  iteration time= 0.21261143684387207
Number of iterations = 2 tolerance = 5.11501113439e-07  iteration time= 0.18707823753356934
max ddh =  0.00044589700976
step = 137  avg d_h= 0.232491019512
Number of iterations = 1 tolerance = 0.000218850974336  iteration time= 0.1929934024810791
Number of iterations = 2 tolerance = 5.98418210825e-07  iteration time= 0.18689632415771484
max ddh =  0.000444620485084
step = 138  avg d_h= 0.23248768263
Number of iterations = 1 tolerance = 0.000218290444937  iteration time= 0.21539855003356934
Number of 

...

Number of iterations = 2 tolerance = 6.85413673631e-08  iteration time= 0.19579792022705078
max ddh =  0.000185936908324
step = 843  avg d_h= 0.230093731936
Number of iterations = 1 tolerance = 8.03927731132e-05  iteration time= 0.18894195556640625
Number of iterations = 2 tolerance = 6.95704457379e-08  iteration time= 0.19693541526794434
max ddh =  0.000186020302814
step = 844  avg d_h= 0.230087947535
Number of iterations = 1 tolerance = 8.0427208556e-05  iteration time= 0.19139552116394043
Number of iterations = 2 tolerance = 7.0624162655e-08  iteration time= 0.19883012771606445
max ddh =  0.000186103892821
step = 845  avg d_h= 0.230082153664
Number of iterations = 1 tolerance = 8.04622844216e-05  iteration time= 0.1909956932067871
Number of iterations = 2 tolerance = 3.29938704691e-08  iteration time= 0.19730830192565918
max ddh =  0.000186187531871
step = 846  avg d_h= 0.230076350337
Number of iterations = 1 tolerance = 8.04730583319e-05  iteration time= 0.1915593147277832
Number o

Number of iterations = 2 tolerance = 1.81595911076e-07  iteration time= 0.19311213493347168
max ddh =  0.000188777704499
step = 877  avg d_h= 0.229891837015
Number of iterations = 1 tolerance = 8.11550132691e-05  iteration time= 0.19161725044250488
Number of iterations = 2 tolerance = 3.6048783231e-07  iteration time= 0.19008398056030273
max ddh =  0.000188867174469
step = 878  avg d_h= 0.22988573889
Number of iterations = 1 tolerance = 8.121200552e-05  iteration time= 0.18483686447143555
Number of iterations = 2 tolerance = 2.95070425595e-07  iteration time= 0.19308805465698242
max ddh =  0.000188957542329
step = 879  avg d_h= 0.229879631804
Number of iterations = 1 tolerance = 8.12775573117e-05  iteration time= 0.18347954750061035
Number of iterations = 2 tolerance = 2.47458596582e-07  iteration time= 0.18994760513305664
max ddh =  0.00018904894286
step = 880  avg d_h= 0.229873515773
Number of iterations = 1 tolerance = 8.13522279123e-05  iteration time= 0.1875760555267334
Number of 

Number of iterations = 1 tolerance = 8.08034946142e-05  iteration time= 0.2012639045715332
Number of iterations = 2 tolerance = 2.6796630924e-07  iteration time= 0.1908552646636963
max ddh =  0.000192776863533
step = 911  avg d_h= 0.229679573033
Number of iterations = 1 tolerance = 8.0465194473e-05  iteration time= 0.19582605361938477
Number of iterations = 2 tolerance = 2.45023519169e-07  iteration time= 0.1923060417175293
max ddh =  0.000192842692949
step = 912  avg d_h= 0.229673179462
Number of iterations = 1 tolerance = 8.01450063077e-05  iteration time= 0.19559025764465332
Number of iterations = 2 tolerance = 2.25041683865e-07  iteration time= 0.18822336196899414
max ddh =  0.000192904003767
step = 913  avg d_h= 0.229666777483
Number of iterations = 1 tolerance = 7.98405018674e-05  iteration time= 0.19627642631530762
Number of iterations = 2 tolerance = 2.07511726604e-07  iteration time= 0.19289708137512207
max ddh =  0.000192961087742
step = 914  avg d_h= 0.229660367116
Number of

Number of iterations = 1 tolerance = 7.33745428514e-05  iteration time= 0.19820690155029297
Number of iterations = 2 tolerance = 4.24670493894e-08  iteration time= 0.18957877159118652
max ddh =  0.000193453261946
step = 945  avg d_h= 0.229457583516
Number of iterations = 1 tolerance = 7.32028133506e-05  iteration time= 0.1947021484375
Number of iterations = 2 tolerance = 4.09757818648e-08  iteration time= 0.18803024291992188
max ddh =  0.000193442754831
step = 946  avg d_h= 0.229450914332
Number of iterations = 1 tolerance = 7.30318803337e-05  iteration time= 0.1954500675201416
Number of iterations = 2 tolerance = 3.9558656657e-08  iteration time= 0.20287609100341797
max ddh =  0.000193431162724
step = 947  avg d_h= 0.229444237368
Number of iterations = 1 tolerance = 7.28617145574e-05  iteration time= 0.20305514335632324
Number of iterations = 2 tolerance = 3.84169295611e-08  iteration time= 0.18930387496948242
max ddh =  0.000193418519352
step = 948  avg d_h= 0.229437552645
Number of 

Number of iterations = 1 tolerance = 6.97567041568e-05  iteration time= 0.19440913200378418
Number of iterations = 2 tolerance = 3.90708719336e-08  iteration time= 0.19050145149230957
max ddh =  0.000192647243484
step = 979  avg d_h= 0.229226583516
Number of iterations = 1 tolerance = 6.98214454905e-05  iteration time= 0.19379353523254395
Number of iterations = 2 tolerance = 3.90842946184e-08  iteration time= 0.19098210334777832
max ddh =  0.000192613720189
step = 980  avg d_h= 0.229219660762
Number of iterations = 1 tolerance = 6.98863934045e-05  iteration time= 0.1915571689605713
Number of iterations = 2 tolerance = 3.90972202929e-08  iteration time= 0.19532299041748047
max ddh =  0.000192579835183
step = 981  avg d_h= 0.229212730891
Number of iterations = 1 tolerance = 6.99515454168e-05  iteration time= 0.1943047046661377
Number of iterations = 2 tolerance = 3.91096912182e-08  iteration time= 0.19342398643493652
max ddh =  0.000192545601517
step = 982  avg d_h= 0.229205793924
Number

In [11]:
figure()
imshow_grid_at_node(mg, h)
title('head')

figure()
plot_links(mg, 'conduit__discharge')#,logvalues=True)
title('discharge')


figure()
plot_links(mg, 'hydraulic__diameter')#,logvalues=True)
title('d_H')


Text(0.5, 1.0, 'd_H')

In [17]:
figure()
plot(mg.at_link['conduit__discharge'][mg.active_links])

In [16]:
mc.run_one_step()
pfn.run_one_step()

#mc = MeltCreep(mg, dt=5000.)


max ddh =  4.545746571648965e-05
Number of iterations = 1 tolerance = 6.646520870778883e-05
Number of iterations = 2 tolerance = 8.926521188890035e-09


In [12]:
mg.calc_net_flux_at_node(Q, out=net_node_flux)
net_Q = mg.calc_net_flux_at_node(Q)/100. - mg.at_node['input__discharge'] 
figure()
imshow_grid_at_node(mg, net_Q)

NameError: name 'net_node_flux' is not defined

In [20]:
mg.at_node['input__discharge']

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [24]:
from scipy.optimize import fsolve
def network_residuals(heads):
    h[mg.core_nodes] = heads
    mg.calc_grad_at_link(h, out=dhdx)
    Q = np.sign(dhdx)*np.sqrt(np.fabs(dhdx))
    return mg.calc_net_flux_at_node(Q, out=net_node_flux)[mg.core_nodes]/mg.dx - mg.at_node['input__discharge'][mg.core_nodes]
#This function doesn't account for inflow at nodes


In [25]:
mc.run_one_step()
mg.calc_grad_at_link(h, out=dhdx)
mg.at_node['hydraulic__head'][mg.core_nodes] = fsolve(network_residuals, h[mg.core_nodes])
mg.at_link['conduit__discharge'][mg.core_nodes] = np.sign(dhdx[mg.core_nodes])*np.sqrt(np.fabs(dhdx[mg.core_nodes]))

print "mean h =", mean(mg.at_node['hydraulic__head'][mg.core_nodes] )
print "mean Q =", mean(Q)

mean ddh =  -5.66192997218e-18
mean h = 166.222550077
mean Q = 0.0865514465643


In [22]:
mg.dx

100.0

In [26]:
figure()
imshow_grid_at_node(mg, h)

In [24]:
figure()
plot_links(mg, 'hydraulic__diameter', autoscale=False, vmin=0, vmax=1.)
print mean(mg.at_link['hydraulic__diameter'])

3.50074727364e+69


In [56]:
figure()
plot_links(mg, 'conduit__discharge', autoscale=True, magnitude=True)
#print mean(mg.at_link['hydraulic__diameter'])

In [10]:
mc = MeltCreep(mg, dt=5000.)
#mean(mc.Q)

In [17]:
plot(mg.at_link['hydraulic__diameter'][200:300])


Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902


Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.6482677381

In [19]:
#figure()

pfn.run_one_step()
mc.run_one_step()

Number of iterations = 1 tolerance = 0.0031818002267
Number of iterations = 2 tolerance = 1.26436621244e-05
mean ddh =  -6.86035808489e-06


In [41]:
#for m in mc.melt:
#    print m
print max(mc.melt[~np.isnan(mc.melt)])
print max(mc.creep)
#print mc.d_h.mean()
#print mg.at_link['hydraulic__diameter']

3.35041761374e-06
2.12710591355e-06


In [15]:
#mc.run_one_step()

mc._grid.at_link['conduit__discharge']

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])